In [1]:
def basicstats(values):
    '''
    basic statistics over the list of values
    returns a list with min,q1,median,q3,max
    '''
    #var=0.0
    #mean=0.0
    n=len(values)
    #minmax=[min(values),max(values)]
    #for item in values:
    #    mean+=item
    #    var+=item**2
    
    #mean/=n
    #var=var/n-mean**2
    
    ordered=sorted(values,reverse=False)
    quant=[.25,.5,.75]
    q=[min(values)]
    for prop in quant:
        
        place=n*prop
        arm=place-int(place)
        place=int(place)
        q.append(arm*ordered[place+1]+(1-arm)*ordered[place])
    q.append(max(values))
    return q
   

In [2]:
#LOMCE

def import_rubs_LOMCE(nomfich):
    '''
    import the LOE rubs
    returns a dictionary where keys are nom alumns and values are another dictionary
    where the keys are the rubs and the values % over valid opinions 
    '''
    import re
    
    try:
        fich=open(nomfich+'.csv.0','r')
        cont=fich.readlines()
        fich.close()
    except:
        print 'No se encuentra el fichero %s' % nomfich+'.csv.0'
        return {}

    data=[]
    quest=[]
    nump=0
    ok=False
    count=0

    #Load data
    for aux in cont:
        item= aux.strip().split(',')
        if 'Encuestas:' in item[0]:
            nump=item[0].split(':')[1]
            nump=int(nump.strip('"'))
        elif 'Pregunta' in item[0] and 'Respuesta' in item[1]:
            ok=True
        elif ok:
            if not count % 3:
                cad=item[0].strip('"')+','+item[1].strip('"')
                temp=[re.sub(' ,',',',cad)]
                if count<1:
                    quest=[dummy.strip('"') for dummy in item[2:]]
            elif count %3 ==1:
                temp+=[int(dummy) for dummy in item[1:]]
                data.append(temp)
            count+=1

    #Process data            
    quest[0]='num.ops.'
    odata=[]
    for item in data:
        item[1]=nump-item[1]
        for key in range(2,len(item)):
            item[key]=1.0*item[key]/item[1]
        odata.append(item)

    

    leng=range(len(odata[1][2:]))
    quest=['num.ops','RAZOA', 'AFRONTA', 'RESOLVE', 'PARTICIPA', 'AUTONOMO', 'PRESENTA', 'DIXITAL', 'EQUIPO', 'COMPRENDE', 'REDACTA', 'PLANIFICA', 'RESPECTA', 'ACADA','MELLORA','EMPEORA']
    result={}
    
    for item in odata:
        cad=re.sub('^ ','',item[0].strip())
        result[cad]={key:value for key,value in zip(quest[:len(item[1:])],item[1:])}
        
    
    return result

In [3]:
#import rubs LOE
def import_rubs_LOE(nomfich):
    '''
    import the LOE rubs
    returns a dictionary where keys are nom alumns and values are another dictionary
    where the keys are the rubs and the values % over valid opinions 
    '''
    import re
    
    try:
        fich=open(nomfich+'.csv.1','r')
        cont=fich.readlines()
        fich.close()
    except:
        print 'No se encuentra el fichero %s' % nomfich+'.csv.1'
        return {}

    keys=['COMPRENSIÓN','EXPRESIÓN','CLASE','CASA','PROBAS','TÉCNICAS','DISRUPTIVA','EQUIPO','MELLORA','EMPEORA']

    aux=cont.pop(0)
    alumn=[dummy for dummy in aux.split('"') if dummy !=',']
    #alumn=tuple(dummy.decode('utf-8') for dummy in alumn[4:-2])
    alumn=tuple(dummy for dummy in alumn[4:-2])
    alumns={}
    for item in alumn:
        alumns[item]={dummy:[] for dummy in keys}


    import re
    cont=[re.sub('\\n','',dummy) for dummy in cont]
    cad=''
    for item in cont:
        if item[0]==',':
            if cad: 
                opinions=[dummy.strip('"') for dummy in cad.split(',')[2:-2] ]
                doc=opinions.pop(0)
                for opinion,alumno in zip(opinions,alumn):
                    if opinion:
                        for meti in opinion.split('.'):
                            if meti:
                                for key in keys:
                                    if key in meti:
                                        alumns[alumno][key].append(doc)
                                        break



            cad=item

        else: cad+='.'+item

    if cad: 
        opinions=[dummy.strip('"') for dummy in cad.split(',')[2:-2] ]
        doc=opinions.pop(0)
        for opinion,alumno in zip(opinions,alumn):
            if opinion:
                for meti in opinion.split('.'):
                    if meti:
                        for key in keys:
                            if key in meti:
                                alumns[alumno][key].append(doc)
                                break

    #LOE
    #Process data      

    odata=[]
    quest=[dummy.decode('utf-8') for dummy in keys]
    leng=len(quest)
    for item in alumn:
        opins=set()
        result=[item,0]
        for key in keys:
            rol=alumns[item][key]
            result.append(len(rol))
            for doc in rol:
                opins.add(doc)
        if opins:
            result[1]=len(opins)
            for key in range(leng):
                result[key+2]=1.0*result[key+2]/result[1]
        odata.append(result)

    result={}
    quest=['num.ops']+keys
    for alumn in odata:
        #aux=alumn[0].split(',')
        #cad=aux[1]+' '+aux[0].strip()
        #cad=re.sub('^ ','',cad)
        cad=re.sub(' ,',',',alumn[0])
        result[cad]={key:value for key,value in zip(quest,alumn[1:])}
        
    return result


In [4]:
def nomtohtml(n):
    '''
    converts utf names to html names
    '''
    from re import sub
    
    acute=[
        ['á','a'],['Á','A'],['é','e'],['É','E'],['í','i'],['Í','I'],['ó','o'],['Ó','O'],['ú','u'],['Ú','U']
    ]
    tilde=[
        ['ñ','n'],['Ñ','N']
    ]
    print n
    for item in acute:
        n=sub(item[0],'&'+item[1]+'acute;',n)
    print n
    for item in tilde:
        n=sub(item[0],'&'+item[1]+'tilde;',n)
    print n
    return n

In [5]:
def barchart(data,axis,name,lines=True,rot=0,xlabel='',ylabel='',facum=False):
    '''
    Makes a barchar plot from data and stores it in a png file
    axis: X axis tick marks
    name: name of file to save the figure
    lines: if true draw horiz lines at 0.25, 0.5 and 0.75
    rot: rotation of x tick marks
    xlabel: X axis label
    ylabel: Y axis label
    facum: if true adds an accumulative frequency line
    '''
    import matplotlib as mpl 
    import matplotlib.pyplot as plt
    
    width=0.4
    # agg backend is used to create plot as a .png file
    mpl.use('agg')

    fig = plt.figure(1, figsize=(7, 4))

    # Create an axes instance
    ax = fig.add_subplot(111)
    
    #draw horiz line
    if lines:
        ax.axhline(y=0.25,linewidth=2, color='green',linestyle=':',alpha=0.5)
        ax.axhline(y=0.5,linewidth=2, color='blue',linestyle=':',alpha=0.5)
        ax.axhline(y=0.75,linewidth=2, color='red',linestyle=':',alpha=0.5)
        
        
    if facum:
        acum=[data[0]]
        for value in data[1:]:
            acum.append(acum[-1]+value)
        
        for index in range(len(axis)-1):
            
            ax.plot([int(axis[index])+width,int(axis[index+1])+width],[acum[index],acum[index+1]],'g-')
    
    bc=ax.bar(range(len(data)),data, color='white', edgecolor='black', linewidth=2)
    
    ax.set_xlim(-width,len(data)+width)
    if not facum:
        ax.set_ylim(0,max(data)*1.1)
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    
    xTickMarks = axis
    ax.set_xticks([dummy+width for dummy in range(len(data))])
    xtickNames = ax.set_xticklabels(xTickMarks)
    plt.setp(xtickNames, rotation=rot, fontsize=10)
    
    fig.savefig(name+'.png', bbox_inches='tight')
    plt.close('all')

In [6]:
def boxplot(data,name):
    '''
    Makes a boxplot from data and stores it in a png file
    name: name of file to save the figure
    '''
    import matplotlib as mpl 
    import matplotlib.pyplot as plt 
    
    # agg backend is used to create plot as a .png file
    mpl.use('agg')

    fig = plt.figure(2, figsize=(7, 4))

    # Create an axes instance
    ax = fig.add_subplot(111)
    
    #draw horiz line
    ax.axhline(y=5,linewidth=2, color='r',linestyle=':',alpha=0.5)
    
    # Create the boxplot
    bp = ax.boxplot(data, patch_artist=True)
    
    ## change outline color, fill color and linewidth of the boxes
    for box in bp['boxes']:
        # change outline color
        #box.set( color='#7570b3', linewidth=2)
        box.set( color='black',linewidth=2)
        # change fill color
        #box.set( facecolor = '#1b9e77' )
        box.set( facecolor = 'white' )

    ## change color and linewidth of the whiskers
    for whisker in bp['whiskers']:
        whisker.set(color='#7570b3', linewidth=2)

    ## change color and linewidth of the caps
    for cap in bp['caps']:
        cap.set(color='#7570b3', linewidth=2)

    ## change color and linewidth of the medians
    for median in bp['medians']:
        #median.set(color='#b2df8a', linewidth=2)
        median.set(color='black', linewidth=2)

    ## change the style of fliers and their fill
    for flier in bp['fliers']:
        flier.set(marker='o', color='#e7298a', alpha=0.5)
    plt.xlabel('Alumnos')
    plt.ylabel('Nota')
    # Save the figure
    fig.savefig(name+'.png', bbox_inches='tight')
    plt.close('all')

In [7]:
def procesa_descriptiva(curso,nom_curso,evaluac=1):
    '''
    Generates pdf reports for group 
    curso: dictionary with students names and his performance 
    nom_curso: group denomination
    evaluacion: trimester considered
    '''


    #proceso datos curso
    
    alumns=sorted(curso.keys())
    nomfichs=[]
    numpag=0
    
    nomfich=PATH+nom_curso+'_'+str(numpag)+'.html'
    fich=open(nomfich,'w')
    nomfichs.append(nomfich)
    if 'eso' in nom_curso:
        label_curso=nom_curso[0]+'&deg;ESO-'+nom_curso[-1].upper()
    elif 'bac' in nom_curso:
        label_curso=nom_curso[0]+'&deg;Bac.-'+nom_curso[-1].upper()
    else:
        label_curso=nom_curso
    fich.write('<html>\n')
    fich.write('<head>\n <title>%s %i&ordf; aval.</title>\n </head>\n' % (label_curso,evaluac))
    fich.write('<body>\n')
    fich.write('<h2><center>%s %i&ordf; avaliaci&oacute;n</center></h2>\n ' % (label_curso,evaluac))
    numasig=1
    for alumn in alumns:
        numasig=max(numasig,len(curso[alumn][0]))
    notas=[]
    resalum=[]
    #print alumns
    #print numasig
    numsusp=[0]*(numasig+1)
    for alumn in alumns:
        resalum.append(curso[alumn])
        notasalum=curso[alumn][0].values()
        notas+=notasalum
        #print notasalum
        ns=sum([1 for dummy in notasalum if dummy<5])
        numsusp[ns]+=1

    while not numsusp[-1]:
        numsusp.pop(-1)
    stats=basicstats(notas)
    frecs=[0]*10
    for nota in notas:
        frecs[nota-1]+=1


    data=[dummy[0].values() for dummy in resalum]
    nomfich=PATH+'boxplot_grup'
    boxplot(data,nomfich)
    fich.write('<BR>\n')
    fich.write('<center>Boxplot das notas dos alumnos, ordeados alfabeticamente</center>\n')
    fich.write('<BR>\n')
    nomfich+='.png'
    fich.write('<img src="%s"/>\n' % (nomfich))

    '''
    fich.write('<BR>\n')

    fich.write('<h3><center>Estadisticas de las notas del grupo </center></h3>\n')
    labels_estad=['Q1 :','mediana :','Q3 :']
    fich.write('<BR>\n')
    fich.write('<table>\n')
    for label,value in zip(labels_estad,stats[1:-1]):
        fich.write('<tr>\n')
        fich.write('<td>%s</td><td>%g</td>\n' % (label,value))
        fich.write('</tr>\n')
    fich.write('</table>\n')
    '''

    tot=sum(numsusp)
    data=[dummy*1.0/tot for dummy in numsusp]
    axis=range(len(numsusp))

    labels_estad=[str(dummy) for dummy in axis]
    labels_estad.insert(0,'<b>num.susp.</b>')
    fich.write('<BR>\n')
    fich.write('<center><table border="1px" cellpadding="10px">\n')
    fich.write('<tr>\n')
    for label in labels_estad:
        fich.write('<td align="center">%s</td>' % (label))
    fich.write('\n</tr>\n')
    labels_estad=[str(dummy) for dummy in numsusp]
    labels_estad.insert(0,'<b>num.alumn.</b>')
    fich.write('<tr>\n')
    for label in labels_estad:
        fich.write('<td align="center">%s</td>' % (label))
    fich.write('\n</tr>\n')
    fich.write('</table></center>\n')

    nomfich=PATH+'barchart_grup_notas'

    barchart(data,axis,nomfich,lines=True,rot=0,xlabel='num.suspensos',ylabel='prop.alumnos',facum=True)
    #fich.write('Histograma del numero de suspensos por alumno')
    #fich.write('<BR>\n')
    nomfich+='.png'
    fich.write('<img src="%s"/>\n' % (nomfich))
    fich.write('</body>\n</html>\n')

    fich.close()
    #proceso resultados alumno

    for alumn in alumns:
        numpag+=1
        aux=alumn.strip().split(',')
        nom_alumn=aux[1]+' '+aux[0]
        nom_alumn=nomtohtml(nom_alumn)
        nomfich=PATH+nom_curso+'_'+str(numpag)+'.html'
        fich=open(nomfich,'w')
        nomfichs.append(nomfich)
        notas=curso[alumn][0]
        rubs=curso[alumn][1]
        fich.write('<html>\n')
        fich.write('<head>\n <title>%s %s</title>\n </head>\n' % (label_curso,nom_alumn))
        fich.write('<body>\n')
        fich.write('<h3><center>%s</center></h3>\n ' % (nom_alumn))
        suspensos=[]
        for key,value in notas.iteritems():
            if value<5:
                suspensos.append([key,value])

        if len(suspensos):
            fich.write('<b>n&deg; suspensos: %i</b>\n' % len(suspensos))
            suspensos.sort(key=lambda x:x[1])
            fich.write('<br>\n')
            for item in suspensos:
                fich.write('%s (%i)  ' % tuple(item))
            fich.write('<br>\n')
        frecnotas=[0]*10
        for value in notas.values():
            frecnotas[value-1]+=1
        nomfich=PATH+'notas_%i' % numpag
        barchart(frecnotas,[str(i) for i in range(1,11)],nomfich,lines=False,rot=0,xlabel='nota',ylabel='frec.',facum=False)
        nomfich+='.png'
        fich.write('<img src="%s"/>\n' % (nomfich))
        fich.write('<br>\n')
        if rubs:
            lrubs=sorted(rubs.keys())
            lrubs.remove('num.ops')
            if 'MELLORA' in lrubs:
                lrubs.remove('MELLORA')
                lrubs.append('MELLORA')
            if 'EMPEORA' in lrubs:
                lrubs.remove('EMPEORA')
                lrubs.append('EMPEORA')
            vrubs=[rubs[key] for key in lrubs]
            indices=[]
            for index,value in enumerate(vrubs):
                if not value:
                    indices.append(index)
            indices.reverse()
            for index in indices:
                lrubs.pop(index)
                vrubs.pop(index)
            lrubs=[dummy.decode('utf-8') for dummy in lrubs]
            numops=rubs['num.ops']
            if numops:
                fich.write('<b><center>Rubricas</center></b>')
                nomfich=PATH+'rubs_%i' % numpag
                barchart(vrubs,lrubs,nomfich,lines=True,rot=45,xlabel='',ylabel='prop.',facum=False)
                nomfich+='.png'
                fich.write('n&deg; opini&oacute;ns: %i\n' % numops)
                fich.write('<br>\n')
                fich.write('<img src="%s"/>\n' % (nomfich))



        fich.write('</body>\n</html>\n')

        fich.close()
    #generar el pdf con wkhtmltopdf
    import os

    comando='~/Descargas/wkhtmltox/bin/wkhtmltopdf '+' '.join(nomfichs)+' '+PATH+'%s.pdf' % nom_curso
    
    os.system(comando)
    comando='rm -f '+PATH+'*.html'
    os.system(comando)
    comando='rm -f '+PATH+'*.png'
    os.system(comando)

In [8]:
def jsonice(data,name_fich):
    '''
    Funcion para guardar y recuperar datos previamente procesados.
    name_fich: nombre del fichero de texto (con PATH y sin extension) para leer o escribir datos 
    Si data es un objeto python valido sera guardado como estructura json en el fichero de txto name_fich.json 
    En este caso la función devuelve 0 si la operación tiene exito, 1 en caso contrario
    Si data es None, se recupera la estructura de name_fich.json y la función devuelve el objeto recreado en caso de  exito, 1 en caso contrario.
    '''
    import json
    if not data:
        try:
            fich=open(name_fich+'.json','r')
            data=json.load(fich)
            fich.close()
            return data
        except IOError:
            print 'No se puede abrir %s' % name_fich
            return 1
        except:
            print 'No se puede procesar %s' % name_fich
            return 1
    else:
        try:
            fich=open(name_fich+'.json','w')
            json.dump(data,fich)
            fich.close()
            return 0
        except IOError:
            print 'No se puede abrir %s' % namefich
            return 1
        except:
            print 'no se puede procesar los datos'
            return 1
        

In [30]:
def procesa_descriptiva_multiple(curso,nom_curso,evaluac=[1],okrubs=True):
    '''
    Generates pdf reports for group 
    curso: list with one dictionary for trimester with students names and his performance 
    nom_curso: group denomination
    evaluacion: trimester considered
    '''


    #proceso datos curso
    alumns=set(curso[0].keys())
    numtrim=len(curso)
    rangetrim=range(numtrim)
    for t in (1,numtrim-1):
        alumns=alumns.intersection(curso[t].keys())
    alumns=sorted(list(alumns))
    nomfichs=[]
    numpag=0
    
    nomfich=PATH+nom_curso+'_'+str(numpag)+'.html'
    fich=open(nomfich,'w')
    nomfichs.append(nomfich)
    if 'eso' in nom_curso:
        label_curso=nom_curso[0]+'&deg;ESO-'+nom_curso[-1].upper()
    elif 'bac' in nom_curso:
        label_curso=nom_curso[0]+'&deg;Bac.-'+nom_curso[-1].upper()
    else:
        label_curso=nom_curso
    fich.write('<html>\n')
    saval=','.join([str(val)+'&ordf' for val in evaluac])
    fich.write('<head>\n <title>%s %s; aval.</title>\n </head>\n' % (label_curso,saval))
    fich.write('<body>\n')
    fich.write('<h2><center>%s %s; avaliaci&oacute;n</center></h2>\n ' % (label_curso,saval))
    numasig=1
    for alumn in alumns:
        for t in rangetrim:
            numasig=max(numasig,len(curso[t][alumn][0]))
    notas=[[] for dummy in rangetrim]
    resalum=[[] for dummy in rangetrim]
    numsusp=[[0]*(numasig+1) for dummy in rangetrim]
    maxsusp=0
    for alumn in alumns:
        for t in rangetrim:
            resalum[t].append(curso[t][alumn])
            notasalum=curso[t][alumn][0].values()
            notas[t]+=notasalum
            ns=sum([1 for dummy in notasalum if dummy<5])
            numsusp[t][ns]+=1
            maxsusp=max(maxsusp,ns)

    for t in rangetrim:
        numsusp[t]=numsusp[t][:maxsusp+1]
    stats=[basicstats(notass) for notass in notas] 
    frecs=[[0]*11 for dummy in rangetrim]
    for t in rangetrim:
        for nota in notas[t]:
            frecs[t][nota]+=1


    data=[]
    for t in rangetrim:
        tot=sum(numsusp[t])
        data.append([dummy*1.0/tot for dummy in numsusp[t]])
    axis=range(maxsusp+1)

    labels_estad=[str(dummy) for dummy in axis]
    labels_estad.insert(0,'<b>num.susp.</b>')
    fich.write('<BR>\n')
    fich.write('<center><table border="1px" cellpadding="10px">\n')
    fich.write('<tr>\n')
    for label in labels_estad:
        fich.write('<td align="center">%s</td>' % (label))
    fich.write('\n</tr>\n')
    for t in rangetrim:
        labels_estad=[str(dummy) for dummy in numsusp[t]]
        labels_estad.insert(0,'<b>n&deg.alumn.-%i&ordf;av.</b>'%(evaluac[t]))
        fich.write('<tr>\n')
        for label in labels_estad:
            fich.write('<td align="center">%s</td>' % (label))
        fich.write('\n</tr>\n')
    fich.write('</table></center>\n')

    nomfich=PATH+'barchart_grup_notas'
    legtex=[u'%iª av.'% value for value in evaluac]
    barchart_multiple(data,axis,legtex,nomfich,lines=True,rot=0, xlabel='num.suspensos', ylabel='prop.alumnos', facum=True)
    #fich.write('Histograma del numero de suspensos por alumno')
    #fich.write('<BR>\n')
    nomfich+='.png'
    fich.write('<img src="%s"/>\n' % (nomfich))
    fich.write('</body>\n</html>\n')

    fich.close()
    #proceso resultados alumno
    
    
    
    
    for alumn in alumns:
        numpag+=1
        aux=alumn.strip().split(',')
        nom_alumn=aux[1]+' '+aux[0]
        nom_alumn=unicode(nom_alumn)
        nom_alumn=nom2html(nom_alumn)
        
        nomfich=PATH+nom_curso+'_'+str(numpag)+'.html'
        fich=open(nomfich,'w')
        nomfichs.append(nomfich)
        notas=[]
        rubs=[]
        for t in rangetrim:
            notas.append(curso[t][alumn][0])
            rubs=curso[t][alumn][1]
        
        fich.write('<html>\n')
        fich.write('<head>\n <title>%s %s</title>\n </head>\n' % (label_curso,nom_alumn))
        fich.write('<body>\n')
        fich.write('<h3><center>%s</center></h3>\n ' % (nom_alumn))
        suspensos=[[] for dummy in rangetrim]
        for t in rangetrim:
            for key,value in notas[t].iteritems():
                if value<5:
                    suspensos[t].append([key,value])

            if len(suspensos[t]):
                fich.write('<b>n&deg; suspensos %i&ordf; av.: %i</b>\n' % (evaluac[t],len(suspensos[t])))
                suspensos[t].sort(key=lambda x:x[1])
                #fich.write('<br>\n')
                for item in suspensos[t]:
                    fich.write('%s (%i)  ' % tuple(item))
                fich.write('<br>\n')
        frecnotas=[[0]*11 for dummy in rangetrim]
        for t in rangetrim:  
            for value in notas[t].values():
                frecnotas[t][value]+=1
        nomfich=PATH+'notas_%i' % numpag
        barchart_multiple(frecnotas,[str(i) for i in range(11)],legtex,nomfich,lines=False,rot=0,xlabel='nota',ylabel='frec.',facum=False)
        nomfich+='.png'
        fich.write('<img src="%s"/>\n' % (nomfich))
        fich.write('<br>\n')
        #Si se procesan rubricas
        if okrubs:
            
            lrubs=set()

            for t in rangetrim:

                if curso[t][alumn][1]:
                    lrubs=lrubs.union(set(curso[t][alumn][1].keys()))
            lrubs=sorted(list(lrubs))
            if 'MELLORA' in lrubs:
                lrubs.remove('MELLORA')
                lrubs.append('MELLORA')
            if 'EMPEORA' in lrubs:
                lrubs.remove('EMPEORA')
                lrubs.append('EMPEORA')
            if 'num.ops' in lrubs:
                lrubs.remove('num.ops')
            vrubs=[[] for dummy in rangetrim]
            for t in rangetrim:
                for rub in lrubs:
                    if rub in curso[t][alumn][1].keys():
                        vrubs[t].append(curso[t][alumn][1][rub])
                    else:
                        vrubs[t].append(0)
            indices=[0]*len(lrubs)

            for t in rangetrim:
                for index,value in enumerate(vrubs[t]):
                    if value>0:
                        indices[index]=1
            popindex=[index for index,value in enumerate(indices) if value==0]
            popindex.sort(reverse=True)   
            for index in popindex:
                lrubs.pop(index)
                for t in range(numtrim):
                    vrubs[t].pop(index)

            if lrubs:
                #lrubs=[dummy.decode('utf-8') for dummy in lrubs]
                lrubs=[unicode(dummy) for dummy in lrubs]
                numops=[curso[t][alumn][1]['num.ops'] if 'num.ops' in curso[t][alumn][1].keys() else 0 for t in rangetrim]

                fich.write('<b><center>Rubricas</center></b>')
                nomfich=PATH+'rubs_%i' % numpag
                legrubs=[legtex[t]+' (%i)'% numops[t] for t in rangetrim]

                barchart_multiple(vrubs,lrubs,legrubs, nomfich, lines=True, rot=45, xlabel='' ,ylabel='prop.' ,facum=False)
                nomfich+='.png'
                #cad=';'.join([str(t+1)+'&ordf;av.' for t in rangetrim])
                #cad+='('+cad+'): ('+','.join([str(numops[t]) for t in rangetrim])+')'
                #fich.write('n&deg; opini&oacute;ns ' +cad)
                #fich.write('<br>\n')
                fich.write('<img src="%s"/>\n' % (nomfich))



        fich.write('</body>\n</html>\n')

        fich.close()
    #generar el pdf con wkhtmltopdf
    import os

    comando='~/Descargas/wkhtmltox/bin/wkhtmltopdf '+' '.join(nomfichs)+' '+PATH+'%s.pdf' % nom_curso
    
    os.system(comando)
    comando='rm -f '+PATH+'*.html'
    os.system(comando)
    comando='rm -f '+PATH+'*.png'
    os.system(comando)

In [10]:
def barchart_multiple(data,axis,legtext,name,lines=True,rot=0,xlabel='',ylabel='',facum=False):
    '''
    Makes a barchar plot from data and stores it in a png file
    data: a list of lists with data to plot. All data shares axis
    axis: X axis tick marks
    legtext: a tuple with the text for legend
    name: name of file to save the figure
    lines: if true draw horiz lines at 0.25, 0.5 and 0.75
    rot: rotation of x tick marks
    xlabel: X axis label
    ylabel: Y axis label
    facum: if true adds an accumulative frequency line
    '''
    import matplotlib as mpl 
    import matplotlib.pyplot as plt
    import numpy as np
    
    #number of data series to plot
    numseries=len(data)
    #colors for data series
    color=['g','b','r']
    width=0.4
    # agg backend is used to create plot as a .png file
    #mpl.use('agg')

    fig = plt.figure(1, figsize=(7, 4))

    # Create an axes instance
    ax = fig.add_subplot(111)
    
    #draw horiz line
    if lines:
        ax.axhline(y=0.25,linewidth=2, color='green',linestyle=':',alpha=0.5)
        ax.axhline(y=0.5,linewidth=2, color='blue',linestyle=':',alpha=0.5)
        ax.axhline(y=0.75,linewidth=2, color='red',linestyle=':',alpha=0.5)
        
        
    if facum:
        
        for numt in range(numseries):
            pdata=data[numt]
            acum=[pdata[0]]
            scol=color[numt]+'-'
            
            for value in pdata[1:]:
                acum.append(acum[-1]+value)

            for index in range(len(axis)-1):

                ax.plot([int(axis[index])+width,int(axis[index+1])+width],[acum[index],acum[index+1]],scol)
                #ax.plot([int(axis[index])+width,int(axis[index+1])+width],[acum[index],acum[index+1]])
    width2=width/numseries
    bc=[[]]*numseries
    ind=np.arange(len(data[0]))
   
    for numt in range(numseries):
        
        bc[numt]=ax.bar(ind+width2*(numt+1),data[numt],width2,color=color[numt], edgecolor='black', linewidth=1)
    
    ax.set_xlim(-width,len(data[0])+width)
    if not facum:
        maxvalue=max([max(dummy) for dummy in data])
        ax.set_ylim(0,maxvalue*1.1)
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    
    legend=[dummy[0] for dummy in bc]
    
    
    #plt.legend(legend,legtext,loc=2)
    plt.legend(legend,legtext,bbox_to_anchor=(0., 1.02, 1., .102), loc=3,ncol=2, mode="expand", borderaxespad=0.)
    xTickMarks = axis
    ax.set_xticks([dummy+width for dummy in range(len(data[0]))])
    xtickNames = ax.set_xticklabels(xTickMarks)
    plt.setp(xtickNames, rotation=rot, fontsize=10)
    
    fig.savefig(name+'.png', bbox_inches='tight')
    #fig.show()
    plt.close('all')

In [11]:
def nom2html(n):
    n=unicode(n)
    sale=''
    for i in n:
        code=ord(unicode(i))
        
        if code==225:
            sale+='&aacute;'
        elif code==233:
            sale+='&eacute;'
        elif code== 237:
            sale+='&iacute;'
        elif code== 243:
            sale+='&oacute;'
        elif code== 250:
            sale+='&uacute;'
        elif code== 193:
            sale+='&Aacute;'
        elif code== 201:
            sale+='&Eacute;'
        elif code== 205:
            sale+='&Iacute;'
        elif code== 211:
            sale+='&Oacute;'
        elif code== 218:
            sale+='&Uacute;'
        elif code== 209:
            sale+='&Ntilde;'
        elif code== 241:
            sale+='&ntilde;'
        elif code==170:
            sale+='&ordf;'
        elif code==186:
            sale+='&deg;'
        else:
            sale+=i
    return sale
        
        

In [12]:
PATH='/home/luis/sustituciones/centros/actual/Moodle/analisis/'


In [18]:
#Importar datos de los alumnos: notas y rubricas
import re

nomfich='notasT3.csv'
fich=open(PATH+nomfich,'r')
raw=fich.readlines()
fich.close
raw.pop(0)
cursos=set()
data=[]
for item in raw:
    aux=item.strip().split(';')
    aux[1]=re.sub('º|\.| ','',aux[1])+aux[2]
    
    aux[1]=aux[1].lower()
    #xua=aux[0].split(',')
    #aux[0]=re.sub('^ ','',xua[1]+' '+xua[0])
    cursos.add(aux[1])
    xua=aux[:2]+aux[3:]
    data.append(xua)
    
dcurso={}
for curso in cursos:
    datac=[dummy for dummy in data if dummy[1]==curso]
    alums=set()
    asigs=set()
    for item in datac:
        alums.add(item[0])
        asigs.add(item[3])
    dcurso[curso]={dummy:{} for dummy in alums}
    if '2eso' in curso or '4eso' in curso:
        drubs=import_rubs_LOE(PATH+curso)
    else:
        drubs=import_rubs_LOMCE(PATH+curso)
    
    for alum in alums:
        result=[dummy for dummy in datac if dummy[0]==alum]
        if alum in drubs.keys():
            rubs=drubs[alum]
        else:
            print curso, alum
            rubs={}
        dcurso[curso][alum]=[{dummy[3]:int(dummy[4]) for dummy in datac if dummy[0]==alum },rubs]
      

#Guarda los datos en estruct json
jsonice(dcurso,PATH+nomfich.split('.')[0])

        

No se encuentra el fichero /home/luis/sustituciones/centros/actual/Moodle/analisis/3esoc.csv.0
3esoc Zahidi, Saad
3esoc Ramilo Moreno, Zaira
3esoc Leiras Pérez, Ruben
3esoc Urgal Carrera, Leticia
3esoc Herrero Giráldez, María
3esoc Álvarez de los Santos, María del Carmen
3esoc García González, Pablo
3esoc Marhab El Hassnaoui, Fadua
3esoc Rodríguez Presa, Manuela
3esoc Rodríguez Domínguez, Carla
3esoc Di Carlo Tedoldi, Anna Daniela
3esoc Martínez Rodríguez, María Erika
3esoc Pérez Otero, Alejandro
3esoc Pérez Alvarez, Albért
3esoc Azulino Pires, Víctor Gabriel
3esoc Fernández Pérez, Alba
3esoc Quijeiro Novás, Raquel
3esoc Iglesias Porto, Ana
3esoc Lorenzo Pereiro, Borja
3esoc Martínez Sio, Miguel
3esoc Pita García, María
3esoc Maquieira Estévez, Jesús
3esoc Rodríguez Lorenzo, David
3esoc Barreiro, Evelyn Magali
3esoc Embuena Gómez, Paula
3esoc Lago Cabaleiro, Aida
3esoc Villalón Alonso, Miguel
3esoc Candán Vidal, Saúl
3esoc Pereyra Borlinke, Valentina
No se encuentra el fichero /home/lu

0

In [12]:
for curso in cursos:
    print curso
    procesa_descriptiva(dcurso[curso],curso,2)

2bacb


/usr/lib/pymodules/python2.7/matplotlib/__init__.py:1173: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


4esoa
3esoc
3esob
3esoa
2esoa
4esoc
4esob
3esod
1esoa
2baca
1esoc
1esob
2esob
2esoc
4esod
1bacc
1bacb
1infofca
2bacc
1baca


In [20]:
dcurso3=jsonice(None,PATH+'resultados/notas3T')
dcurso2=jsonice(None,PATH+'resultados/T2/notas2T')
dcurso1=jsonice(None,PATH+'resultados/T1/notas1T')
comp_curso=set(dcurso2.keys()).intersection(set(dcurso1.keys()))
comp_curso=comp_curso.intersection(set(dcurso3.keys()))
print comp_curso

set([u'3esoc', u'3esob', u'3esoa', u'2esoa', u'4esoc', u'4esob', u'3esod', u'1esoa', u'4esoa', u'1esoc', u'1esob', u'2esob', u'2esoc', u'4esod', u'1bacc', u'1bacb', u'1infofca', u'1baca'])


In [31]:
for curso in comp_curso:
    print curso
    dcurso=[dcurso1[curso],dcurso2[curso],dcurso3[curso]]
    procesa_descriptiva_multiple(dcurso,curso,evaluac=[1,2,3],okrubs=False)

3esoc
3esob
3esoa
2esoa
4esoc
4esob
3esod
1esoa
4esoa
1esoc
1esob
2esob
2esoc
4esod
1bacc
1bacb
1infofca
1baca


In [329]:
nom_alumn

u' Jennifer Rivas Torres'

In [331]:
nom2html(u'Mª del Pilar Ramilo P&eacute;rez')

u'M&ordf; del Pilar Ramilo P&eacute;rez'